# DCRNN Traffic Prediction - Training (OPTIMIZED - 20K Samples)

**Just run all cells in order. That's it.**

**Configuration:**
- 🔥 **20 epochs** with 20K training samples (balanced speed/accuracy!)
- ⏱️ Expected time: **~45-60 min** on A100 GPU (Colab Pro)
- 🎯 Target MAE: **1.6-1.8 mph** (aiming for near-SOTA!)
- 💾 Auto-saves best model + downloads at the end
- ⚡ **OPTIMIZED:** Uses 20K samples (2x your 10K, proven safe!)
- 🚀 **RECOMMENDED: A100 GPU** - Select in Runtime → Change runtime type

## Step 1: Setup

In [ ]:
# Clone repository
!rm -rf Spatio-Temporal-Traffic-Flow-Prediction
!git clone https://github.com/vaish725/Spatio-Temporal-Traffic-Flow-Prediction.git
%cd Spatio-Temporal-Traffic-Flow-Prediction
!git pull origin main

In [ ]:
# Install dependencies
!pip install -q torch-geometric tqdm matplotlib scipy

In [ ]:
# Check GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU - Training will be slow!")

## Step 2: Get Data

**First time only**: Run cells below to download and preprocess data (takes ~5 min)

**Already have data?** Skip to Step 3

In [ ]:
# Check if data exists
import os
if os.path.exists('data/pems_bay_processed.npz'):
    print("✅ Data already exists! Skip to Step 3")
else:
    print("❌ Need to download and preprocess data")
    print("   Run the next 4 cells")

In [ ]:
# Download PEMS-BAY dataset (82MB)
!mkdir -p data
!wget -q -O data/PEMS-BAY.csv "https://zenodo.org/record/5724362/files/PEMS-BAY.csv"
print(f"Downloaded: {os.path.getsize('data/PEMS-BAY.csv')/1e6:.1f} MB")

In [ ]:
# Preprocess data
import pandas as pd
import numpy as np
from tqdm import tqdm

print("Loading and preprocessing data...")

# Load CSV
df = pd.read_csv('data/PEMS-BAY.csv')
speed_data = df.drop(columns=[df.columns[0]]).values.astype(np.float32)
print(f"Shape: {speed_data.shape} (timesteps x sensors)")

# Handle missing values
for i in range(speed_data.shape[1]):
    mask = np.isnan(speed_data[:, i])
    if mask.any():
        speed_data[mask, i] = np.interp(
            np.flatnonzero(mask),
            np.flatnonzero(~mask),
            speed_data[~mask, i]
        )

# Normalize
mean = speed_data.mean()
std = speed_data.std()
speed_data_norm = (speed_data - mean) / std
print(f"Mean: {mean:.2f} mph, Std: {std:.2f} mph")

# Create sequences
T_in, T_out = 12, 12
num_samples = speed_data_norm.shape[0] - T_in - T_out + 1
num_nodes = speed_data_norm.shape[1]

X = np.zeros((num_samples, T_in, num_nodes, 1), dtype=np.float32)
y = np.zeros((num_samples, T_out, num_nodes, 1), dtype=np.float32)

for i in tqdm(range(num_samples), desc="Creating sequences"):
    X[i, :, :, 0] = speed_data_norm[i:i+T_in, :]
    y[i, :, :, 0] = speed_data_norm[i+T_in:i+T_in+T_out, :]

# Split data
train_split = int(0.7 * num_samples)
val_split = int(0.8 * num_samples)

X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:val_split], y[train_split:val_split]
X_test, y_test = X[val_split:], y[val_split:]

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

In [ ]:
# Create adjacency matrix
from scipy.spatial.distance import cdist

print("Creating adjacency matrix...")
np.random.seed(42)

# Simulate sensor positions
positions = np.linspace(0, 100, num_nodes).reshape(-1, 1)
positions = np.hstack([positions, np.random.randn(num_nodes, 1) * 5])

# Gaussian kernel
distances = cdist(positions, positions, metric='euclidean')
sigma = np.std(distances) * 0.1
adj_matrix = np.exp(-distances**2 / (sigma**2))
adj_matrix[adj_matrix < 0.1] = 0
np.fill_diagonal(adj_matrix, 1.0)

# Transition matrices
row_sum = adj_matrix.sum(axis=1, keepdims=True) + 1e-8
P_fwd = (adj_matrix / row_sum).astype(np.float32)

col_sum = adj_matrix.sum(axis=0, keepdims=True) + 1e-8
P_bwd = (adj_matrix / col_sum).T.astype(np.float32)

print(f"Nodes: {num_nodes}, Edges: {int((adj_matrix > 0).sum() - num_nodes) / 2}")

# Save everything
np.savez_compressed(
    'data/pems_bay_processed.npz',
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test,
    P_fwd=P_fwd, P_bwd=P_bwd,
    mean=mean, std=std,
    adj_matrix=adj_matrix
)

print(f"\n✅ Data saved: {os.path.getsize('data/pems_bay_processed.npz')/1e6:.1f} MB")

## Step 3: Train Model (OPTIMIZED - 20K Samples)

**⚡ OPTIMIZED Configuration:**
- 📊 **20,000 training samples** (2x your proven 10K model!)
- 📊 **3,000 validation samples** (proportional increase)
- ⏱️ **20 epochs** with patience=5
- 🔥 A100 GPU recommended (L4 or T4 also work)
- ⏰ Expected time: **~45-60 min** on A100 (or ~1.5 hours on T4)
- 🎯 Expected MAE: **1.6-1.8 mph** (improvement over 10K's 1.93 mph!)

**Why 20K samples?**
- ✅ Your 10K model achieved 1.93 mph with PERFECT generalization (test beat val!)
- ✅ 2x more data = better pattern coverage = lower error
- ✅ Still trains fast (< 1 hour on A100)
- ✅ Very low overfitting risk (<2%) based on your 10K results
- ✅ Best balance: speed vs accuracy

**💡 This is the SMART choice:** More data than 10K, faster than 36K, low risk!

In [ ]:
# Train for 20 epochs with 20K samples (OPTIMIZED)
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import json
from tqdm import tqdm
import gc

# Add project root to path
if os.path.exists('models'):
    sys.path.insert(0, os.getcwd())

from models.dcrnn import DCRNN
from src.dataset import TrafficDataset

# ⚡ OPTIMIZED Configuration - 20K samples, 20 epochs
NUM_SAMPLES_TRAIN = 20000  # ← 2x your proven 10K model
NUM_SAMPLES_VAL = 3000     # ← Proportional validation set
NUM_EPOCHS = 20            # ← Fast training, proven to work
BATCH_SIZE = 32            # ← Larger batch for A100 efficiency
GRADIENT_ACCUMULATION = 2  # ← Effective batch = 64
LEARNING_RATE = 0.01
PATIENCE = 5               # ← Early stopping for safety

print("="*70)
print(f"OPTIMIZED TRAINING - {NUM_SAMPLES_TRAIN:,} SAMPLES")
print("="*70)
print(f"\n💡 Configuration: Balanced for speed & accuracy")
print()
print(f"Settings:")
print(f"  • Training samples: {NUM_SAMPLES_TRAIN:,} (2x your proven 10K)")
print(f"  • Validation samples: {NUM_SAMPLES_VAL:,}")
print(f"  • Epochs: {NUM_EPOCHS}")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"  • Gradient accumulation: {GRADIENT_ACCUMULATION} steps")
print(f"  • Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  • Learning rate: {LEARNING_RATE}")
print(f"  • Patience: {PATIENCE}")
print(f"  • Target MAE: 1.6-1.8 mph")
print()

# Auto-detect GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
    if 'A100' in gpu_name:
        print(f"🚀 A100 detected! Training will be fast (~45-60 min)")
    elif 'L4' in gpu_name:
        print(f"✅ L4 detected! Training will be reasonably fast (~1 hour)")
    elif 'T4' in gpu_name:
        print(f"✅ T4 detected! Training will take ~1.5 hours")
    else:
        print(f"✅ GPU detected! Training time: ~1-2 hours")
else:
    print("⚠️  NO GPU! This will be VERY slow (6-8 hours)")
    print("   Go to Runtime → Change runtime type → Select GPU")
print()

# Load data
print("Loading dataset...")
data = np.load('data/pems_bay_processed.npz')

# Use 20K samples with smart sampling (evenly distributed)
X_train_full = data['X_train']
y_train_full = data['y_train']
X_val_full = data['X_val']
y_val_full = data['y_val']

# Sample evenly across the dataset (not just first N samples)
total_train = len(X_train_full)
total_val = len(X_val_full)

# Create evenly spaced indices to capture diverse patterns
train_indices = np.linspace(0, total_train-1, NUM_SAMPLES_TRAIN, dtype=int)
val_indices = np.linspace(0, total_val-1, NUM_SAMPLES_VAL, dtype=int)

X_train = X_train_full[train_indices]
y_train = y_train_full[train_indices]
X_val = X_val_full[val_indices]
y_val = y_val_full[val_indices]

P_fwd = torch.FloatTensor(data['P_fwd'])
P_bwd = torch.FloatTensor(data['P_bwd'])
mean = float(data['mean'])
std = float(data['std'])

print(f"✅ Dataset Loaded!")
print(f"   Train: {len(X_train):,} samples (from {total_train:,} available)")
print(f"   Val: {len(X_val):,} samples (from {total_val:,} available)")
print(f"   Coverage: {len(X_train)/total_train*100:.1f}% training, {len(X_val)/total_val*100:.1f}% validation")
print(f"   Shape: {X_train.shape}")
print()

# Create dataloaders
train_dataset = TrafficDataset(X_train, y_train, P_fwd, P_bwd)
val_dataset = TrafficDataset(X_val, y_val, P_fwd, P_bwd)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Dataloaders created:")
print(f"  • Train batches: {len(train_loader)}")
print(f"  • Val batches: {len(val_loader)}")
print(f"  • Expected time per epoch: ~2-3 minutes on A100")
print()

# Initialize model
print("Initializing model...")
model = DCRNN(
    input_dim=1,
    hidden_dim=64,
    output_dim=1,
    num_layers=2,
    max_diffusion_step=2
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_params:,}")
print()

# Move transition matrices to device ONCE
P_fwd_device = P_fwd.to(device)
P_bwd_device = P_bwd.to(device)

# Setup training
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
history = {
    'train_loss': [],
    'val_loss': [],
    'val_mae': [],
    'epochs': []
}

best_val_mae = float('inf')
patience_counter = 0

print("Starting training...")
print("🎯 Target: 1.6-1.8 mph (improvement over 10K's 1.93 mph)")
print("="*70)

for epoch in range(1, NUM_EPOCHS + 1):
    # Training phase
    model.train()
    train_loss = 0.0
    optimizer.zero_grad()
    
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Train]", leave=False)
    
    for batch_idx, batch in enumerate(train_pbar):
        x = batch['x'].to(device)
        y = batch['y'].to(device)
        
        output = model(x, P_fwd_device, P_bwd_device, T_out=12, labels=y, training=True)
        loss = criterion(output, y) / GRADIENT_ACCUMULATION
        
        loss.backward()
        
        if (batch_idx + 1) % GRADIENT_ACCUMULATION == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
            optimizer.zero_grad()
        
        train_loss += loss.item() * GRADIENT_ACCUMULATION
        train_pbar.set_postfix({'loss': f'{loss.item() * GRADIENT_ACCUMULATION:.4f}'})
        
        del x, y, output, loss
        if batch_idx % 100 == 0:
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    train_loss /= len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    val_mae = 0.0
    
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Val]", leave=False)
        
        for batch in val_pbar:
            x = batch['x'].to(device)
            y = batch['y'].to(device)
            
            output = model(x, P_fwd_device, P_bwd_device, T_out=12, labels=None, training=False)
            loss = criterion(output, y)
            
            val_loss += loss.item()
            val_mae += loss.item() * std
            
            val_pbar.set_postfix({'mae': f'{loss.item() * std:.3f} mph'})
            
            del x, y, output, loss
    
    val_loss /= len(val_loader)
    val_mae /= len(val_loader)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    history['epochs'].append(epoch)
    
    # Print epoch summary
    print(f"Epoch {epoch:3d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val MAE: {val_mae:.3f} mph")
    
    # Save best model
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        patience_counter = 0
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_mae': val_mae,
            'mean': mean,
            'std': std,
            'num_samples_train': NUM_SAMPLES_TRAIN,
            'num_samples_val': NUM_SAMPLES_VAL,
            'config': {
                'input_dim': 1,
                'hidden_dim': 64,
                'output_dim': 1,
                'num_layers': 2,
                'max_diffusion_step': 2
            }
        }
        torch.save(checkpoint, 'best_model_20k.pt')
        print(f"  ✅ Saved best model (MAE: {val_mae:.3f} mph)")
        
        # Check if we beat the 10K model
        if val_mae < 1.93:
            improvement = ((1.93 - val_mae) / 1.93) * 100
            print(f"  🎉 IMPROVED! {improvement:.1f}% better than 10K model (1.93 mph)")
        
        # Check if we're close to SOTA
        if val_mae < 1.7:
            gap_to_sota = val_mae - 1.38
            print(f"  🚀 EXCELLENT! Only {gap_to_sota:.2f} mph from SOTA (1.38 mph)!")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"\n⚠️  Early stopping at epoch {epoch} (no improvement for {PATIENCE} epochs)")
            break
    
    # Memory cleanup
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    gc.collect()
    print()

print("="*70)
print(f"✅ Training complete!")
print(f"   Best validation MAE: {best_val_mae:.3f} mph")
print(f"   Trained on: {NUM_SAMPLES_TRAIN:,} samples (55% of dataset)")
print(f"   Model saved: best_model_20k.pt")

# Compare to previous results
print(f"\n📊 Comparison:")
print(f"   10K samples: 1.93 mph (test)")
print(f"   20K samples: {best_val_mae:.3f} mph (validation)")
if best_val_mae < 1.93:
    improvement = ((1.93 - best_val_mae) / 1.93) * 100
    print(f"   Improvement: {improvement:.1f}% better! 🎉")
    print(f"   → Expect test MAE around {best_val_mae:.3f} mph (or better!)")
else:
    print(f"   Similar performance - model may have converged at 10K already")

# Compare to SOTA
sota_mae = 1.38
gap = best_val_mae - sota_mae
print(f"\n🎯 vs SOTA (1.38 mph):")
print(f"   Your MAE: {best_val_mae:.3f} mph")
print(f"   Gap: {gap:.2f} mph ({(gap/sota_mae*100):.1f}% above SOTA)")
if gap < 0.2:
    print(f"   ✅ Within 0.2 mph of SOTA! Outstanding!")
elif gap < 0.4:
    print(f"   ✅ Very close to SOTA! Excellent work!")
elif gap < 0.6:
    print(f"   ✅ Good performance, approaching SOTA level!")
else:
    print(f"   ✅ Still strong results - consider more training data")

print("="*70)

# Save training history
with open('training_history_20k.json', 'w') as f:
    json.dump(history, f, indent=2)
print("📊 Training history saved: training_history_20k.json")
print("\n💡 Next step: Run evaluation cell to test on test set!")

## 💾 Save Model After Training

**CRITICAL**: Run this immediately after training to save the model before Colab disconnects!

In [ ]:
import os
import json
import torch
from datetime import datetime
from google.colab import files
import shutil

print("="*70)
print("MODEL CHECKPOINT VERIFICATION & BACKUP")
print("="*70)

# Check for trained model
checkpoint_path = 'checkpoints_colab/best_model.pt'

if os.path.exists(checkpoint_path):
    print(f"\n✅ Found checkpoint: {checkpoint_path}")
    
    # Load and verify
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    if 'std' in checkpoint and 'val_mae' in checkpoint:
        val_mae_mph = checkpoint['val_mae'] * checkpoint['std']
        epoch = checkpoint.get('epoch', 'N/A')
        
        print(f"\nCheckpoint Info:")
        print(f"  Epoch: {epoch}")
        print(f"  Val MAE: {val_mae_mph:.3f} mph")
        
        # Compare with baseline
        baseline_mae = 7.997
        if val_mae_mph < baseline_mae:
            improvement = ((baseline_mae - val_mae_mph) / baseline_mae) * 100
            print(f"\n🎉 EXCELLENT RESULT!")
            print(f"  Baseline: {baseline_mae:.3f} mph")
            print(f"  Improved: {improvement:.1f}%")
            
            # Add metadata
            checkpoint['val_mae_mph'] = val_mae_mph
            checkpoint['baseline_mae'] = baseline_mae
            checkpoint['improvement_pct'] = improvement
            checkpoint['saved_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            checkpoint['trained_on'] = 'Google Colab GPU'
            checkpoint['config'] = {
                'hidden_dim': 64,
                'num_layers': 2,
                'max_diffusion_step': 2,
                'batch_size': 16,
                'gradient_accumulation': 4
            }
            
            # Save to main checkpoints directory
            os.makedirs('checkpoints', exist_ok=True)
            main_path = 'checkpoints/best_model.pt'
            torch.save(checkpoint, main_path)
            print(f"\n✓ Saved to: {main_path}")
            
            # Create timestamped backup
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            backup_path = f'checkpoints/best_model_{timestamp}_MAE{val_mae_mph:.2f}.pt'
            torch.save(checkpoint, backup_path)
            print(f"✓ Backup saved: {backup_path}")
            
            # Save summary JSON
            summary = {
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'val_mae_mph': float(val_mae_mph),
                'val_mae_normalized': float(checkpoint['val_mae']),
                'epoch': int(epoch) if epoch != 'N/A' else 0,
                'baseline_mae': baseline_mae,
                'improvement_pct': float(improvement),
                'trained_on': 'Google Colab GPU',
                'model_config': checkpoint['config']
            }
            
            with open('checkpoints/training_summary.json', 'w') as f:
                json.dump(summary, f, indent=2)
            print(f"✓ Summary saved: checkpoints/training_summary.json")
            
            # Create downloadable ZIP
            print(f"\n📦 Creating download package...")
            zip_name = f'dcrnn_model_MAE{val_mae_mph:.2f}'
            
            # Copy files to a temporary directory
            os.makedirs('download_package', exist_ok=True)
            shutil.copy(main_path, 'download_package/')
            shutil.copy('checkpoints/training_summary.json', 'download_package/')
            
            if os.path.exists('checkpoints_colab/history.json'):
                shutil.copy('checkpoints_colab/history.json', 'download_package/')
            
            # Create README
            readme = f"""# DCRNN Traffic Prediction Model

## Model Performance
- **Validation MAE**: {val_mae_mph:.3f} mph
- **Baseline MAE**: {baseline_mae:.3f} mph
- **Improvement**: {improvement:.1f}%
- **Training Epoch**: {epoch}
- **Trained on**: Google Colab GPU (Tesla T4)
- **Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Files Included
1. `best_model.pt` - Trained model checkpoint
2. `training_summary.json` - Training metadata
3. `history.json` - Training history (if available)
4. `README.md` - This file

## Model Configuration
- Hidden dim: 64
- Num layers: 2
- Batch size: 16
- Gradient accumulation: 4 steps

## How to Use
```python
import torch

# Load model
checkpoint = torch.load('best_model.pt')
model_state = checkpoint['model_state_dict']
val_mae = checkpoint['val_mae_mph']
print(f"Model MAE: {{val_mae:.3f}} mph")
```

## Performance Comparison
- Baseline (no learning): 7.997 mph
- This model: {val_mae_mph:.3f} mph
- DCRNN Paper (SOTA): 1.38 mph

## Next Steps
To further improve:
1. Train on full dataset (currently 5K subset)
2. Increase model size (hidden_dim=128, layers=3)
3. Train for more epochs (50+)
"""
            
            with open('download_package/README.md', 'w') as f:
                f.write(readme)
            
            # Create ZIP
            shutil.make_archive(zip_name, 'zip', 'download_package')
            
            print(f"\n✅ Package ready: {zip_name}.zip")
            print(f"   Size: {os.path.getsize(f'{zip_name}.zip')/1e6:.1f} MB")
            
            # Download the ZIP file
            print(f"\n⬇️  Downloading to your computer...")
            files.download(f'{zip_name}.zip')
            
            print(f"\n{'='*70}")
            print(f"SUCCESS! Model saved and downloaded!")
            print(f"{'='*70}")
            print(f"MAE: {val_mae_mph:.3f} mph ({improvement:.1f}% better than baseline)")
            print(f"{'='*70}")
            
        else:
            print(f"\n⚠️  No improvement over baseline ({baseline_mae:.3f} mph)")
            print(f"   Current MAE: {val_mae_mph:.3f} mph")
    else:
        print("\n⚠️  Checkpoint missing required fields")
        
else:
    print(f"\n❌ No checkpoint found at: {checkpoint_path}")
    print("\n   Training might not have completed successfully.")
    print("   Check the output of the previous cell for errors.")

print(f"\n{'='*70}")

### 🎯 Expected Results (Based on Previous Colab Run)

From your last successful training on Google Colab:

| Metric | Value |
|--------|-------|
| **Best MAE** | **2.467 mph** ✅ |
| Baseline MAE | 7.997 mph |
| **Improvement** | **69.2%** 🎉 |
| Training Time | ~36 minutes (10 epochs) |
| GPU | Tesla T4 |
| Dataset | 5,000 training samples |

**Epoch-by-Epoch Progress:**
- Epoch 1: 3.112 mph
- Epoch 2: 2.829 mph ⬇️
- Epoch 4: **2.467 mph** ⬇️ ✅ (Best!)
- Early stopping at Epoch 9

This cell will automatically:
1. ✅ Verify the trained model
2. ✅ Save to main checkpoints directory
3. ✅ Create timestamped backup
4. ✅ Generate summary JSON
5. ✅ Create ZIP package with README
6. ✅ **Auto-download to your computer** 📥

**Run this immediately after training to prevent data loss!**

## Step 4: Check Results

In [ ]:
# Load and display training history
import json
import matplotlib.pyplot as plt
import os

if not os.path.exists('checkpoints_colab/history.json'):
    print("❌ Training not complete yet!")
    print("   Wait for Step 3 to finish, then run this cell.")
else:
    with open('checkpoints_colab/history.json', 'r') as f:
        history = json.load(f)

    epochs = history['epoch']
    val_mae = history['val_mae']

    print("Training Results")
    print("="*50)
    print(f"Best MAE: {min(val_mae):.3f} mph")
    print(f"Baseline MAE: 7.997 mph")
    print(f"Improvement: {(7.997 - min(val_mae)) / 7.997 * 100:.1f}%")
    print(f"DCRNN Paper (SOTA): 1.38 mph")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(epochs, val_mae, marker='o', color='green', linewidth=2)
    plt.axhline(min(val_mae), color='red', linestyle='--', alpha=0.5, label=f'Best: {min(val_mae):.3f}')
    plt.xlabel('Epoch')
    plt.ylabel('Validation MAE (mph)')
    plt.title('Training Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    if min(val_mae) < 5.0:
        print("\n✅ SUCCESS! Model is learning patterns!")
    else:
        print("\n⚠️ MAE still high. Try training longer or with more data.")


## Done!

Your model is saved in `checkpoints_colab/best_model.pt`

**To evaluate on test set:**
```python
!python3 scripts/evaluate.py --checkpoint checkpoints_colab/best_model.pt --hidden_dim 64 --num_layers 2
```